## Loading & Preprocessing data for Actor and Director Based recommendation:

In [33]:
# Consists userID , MovieID and Ratings
import pandas as pd
user_rated = pd.read_csv("user_ratedmovies-timestamps.dat",sep="\t").drop("timestamp",axis=1)
user_rated.head()

,userID,movieID,rating
0,75,3,1.0
1,75,32,4.5
2,75,110,4.0
3,75,160,2.0
4,75,163,4.0


In [35]:
# ActorName , MovieID , Ranking
actor = pd.read_csv("movie_actors.dat",sep="\t",encoding ="latin-1").drop("actorID",axis=1)
actor.head()


,movieID,actorName,ranking
0,1,Annie Potts,10
1,1,Bill Farmer,20
2,1,Don Rickles,3
3,1,Erik von Detten,13
4,1,Greg Berg,17


In [3]:
# User, Age ,Gender
user_demo = pd.read_csv("u.user",sep="|",names=["user_id","age" ,"gender" , "occupation", "zip_code"]).drop(["occupation","zip_code"],axis=1)
user_demo.head()

,user_id,age,gender
0,1,24,M
1,2,53,F
2,3,23,M
3,4,24,M
4,5,33,F


In [4]:
# MErged dataset with User, MovieID, Ratings , age , Gender

urated_udemo = pd.merge(user_rated,user_demo,left_on="userID",right_on="user_id")
urated_udemo.drop("user_id",axis=1,inplace = True)
urated_udemo.head()

,userID,movieID,rating,age,gender
0,75,3,1.0,24,M
1,75,32,4.5,24,M
2,75,110,4.0,24,M
3,75,160,2.0,24,M
4,75,163,4.0,24,M


In [5]:
# Finding the rank 1 actors of all movies and creating a dataframe
import numpy as np
lead_actor = actor[actor["ranking"]==1]
lead_actor.head()

,movieID,actorName,ranking
22,1,Tom Hanks,1
41,2,Robin Williams,1
49,3,Jack Lemmon,1
76,4,Whitney Houston,1
100,5,Steve Martin,1


In [6]:
# Loading movies dataset
movies = pd.read_csv("movies.dat",encoding = "latin-1",sep="\t")
movies = movies.iloc[:,:3].drop(["imdbID"],axis=1)
movies.head()

,id,title
0,1,Toy story
1,2,Jumanji
2,3,Grumpy Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II


In [7]:
# Merged movies and lead_actor
lead_actor_movies = pd.merge(lead_actor,movies,left_on = "movieID",right_on = "id").drop(["id","ranking"],axis=1)

lead_actor_movies["avg_rating"]=lead_actor_movies["movieID"].map(user_rated.groupby("movieID")["rating"].mean())
lead_actor_movies.head()

,movieID,actorName,title,avg_rating
0,1,Tom Hanks,Toy story,3.735154
1,2,Robin Williams,Jumanji,2.976471
2,3,Jack Lemmon,Grumpy Old Men,2.873016
3,4,Whitney Houston,Waiting to Exhale,2.577778
4,5,Steve Martin,Father of the Bride Part II,2.753333


In [8]:
# Merged user_rated and lead_actor_movies
user_rated_lead_movies =pd.merge(user_rated,lead_actor_movies)
user_rated_lead_movies.head()

,userID,movieID,rating,actorName,title,avg_rating
0,75,3,1.0,Jack Lemmon,Grumpy Old Men,2.873016
1,783,3,2.0,Jack Lemmon,Grumpy Old Men,2.873016
2,788,3,3.5,Jack Lemmon,Grumpy Old Men,2.873016
3,1160,3,4.0,Jack Lemmon,Grumpy Old Men,2.873016
4,1174,3,2.5,Jack Lemmon,Grumpy Old Men,2.873016


In [9]:
# Loading director dataset
director = pd.read_csv("movie_directors.dat",encoding="latin",sep = "\t").drop("directorID",axis=1)
director.head()

,movieID,directorName
0,1,John Lasseter
1,2,Joe Johnston
2,3,Donald Petrie
3,4,Forest Whitaker
4,5,Charles Shyer


In [10]:
# this is the final dataframe with all the req features
Final_data = pd.merge(user_rated_lead_movies,director)
Final_data.head()

,userID,movieID,rating,actorName,title,avg_rating,directorName
0,75,3,1.0,Jack Lemmon,Grumpy Old Men,2.873016,Donald Petrie
1,783,3,2.0,Jack Lemmon,Grumpy Old Men,2.873016,Donald Petrie
2,788,3,3.5,Jack Lemmon,Grumpy Old Men,2.873016,Donald Petrie
3,1160,3,4.0,Jack Lemmon,Grumpy Old Men,2.873016,Donald Petrie
4,1174,3,2.5,Jack Lemmon,Grumpy Old Men,2.873016,Donald Petrie


In [11]:
# Dataframe of Fav actors each user
d1= pd.DataFrame(user_rated_lead_movies.groupby("userID")["actorName"].value_counts(ascending =False))
d1 = d1.rename({"actorName":"Number"},axis=1)
d1 = d1.reset_index()
Fav_actor = pd.merge(d1,d1.loc[d1.groupby('userID')["Number"].idxmax(),['userID','actorName']])
Fav_actor.head()


,userID,actorName,Number
0,75,Bruce Willis,13
1,78,Woody Allen,12
2,127,Eddie Murphy,4
3,170,Jim Carrey,5
4,175,Clint Eastwood,7


In [12]:
# Dataframe of Fav director each user
d2= pd.DataFrame(Final_data.groupby("userID")["directorName"].value_counts(ascending =False))
d2 = d2.rename({"directorName":"Number"},axis=1)
d2 = d2.reset_index()
Fav_director = pd.merge(d2,d2.loc[d2.groupby('userID')["Number"].idxmax(),['userID',"directorName"]])
Fav_director.head()


,userID,directorName,Number
0,75,Robert Rodriguez,16
1,78,John Woo,17
2,127,Rob Minkoff,2
3,170,Guy Ritchie,16
4,175,Guy Ritchie,16


## Data Preprocessing for Genre recommendation:

In [13]:
#Reading the dataset containing user details

# Genre Dataset
movie_genres = pd.read_csv("movie_genres.dat",encoding = "latin-1",sep="\t")
movie_genres = movie_genres.pivot("movieID","genre","genre").fillna(0).reset_index()

for i in movie_genres.columns:
    movie_genres[i] = movie_genres[i].replace(i,1)

# Movies name and genre
movies1 = pd.merge(movies,movie_genres,left_on="id" , right_on= "movieID").drop(["movieID"],axis=1)

#merging all the three tables
final_merge1=pd.merge(user_rated,movies1,left_on="movieID",right_on="id")

#dropping repeated columns
final_merge1.drop(["id"],inplace=True,axis=1)


## Data Preprocessing for Search by Actor Name:

In [22]:
import pandas as pd
import numpy as np
import sys

movies4= pd.read_csv("movies.dat",encoding='latin-1',sep='\t')
movies4.drop(['imdbID','spanishTitle','rtID','rtAllCriticsRating','rtAllCriticsNumReviews',\
       'rtAllCriticsNumFresh', 'rtAllCriticsNumRotten', 'rtAllCriticsScore',\
       'rtTopCriticsRating', 'rtTopCriticsNumReviews', 'rtTopCriticsNumFresh',\
       'rtTopCriticsNumRotten', 'rtTopCriticsScore',\
       'rtAudienceNumRatings', 'rtAudienceScore', 'rtPictureURL'],axis=1,inplace=True)
movies4['title'].loc[0]='Toy Story'

movie_actor4= pd.read_csv('movie_actors.dat',encoding='latin-1',sep='\t')
movie_actor4.drop(['actorID'],axis=1,inplace=True)

user_ratedmovies4= pd.read_csv('user_ratedmovies.dat',encoding='latin-1',sep='\t')
user_ratedmovies4.drop(['date_day', 'date_month', 'date_year',\
       'date_hour', 'date_minute', 'date_second'],axis=1,inplace=True)

## Defining Functions for recommendation based on 
   - Actor
   - Director
   - Genre
   - Search By Actor Name

### Actor Function :

In [23]:
def actor_based(UserID):
    # RECOMMENDATION Based On Actors
    # Dataframe of Fav actors each user

    # if any user is not present in the data then we will recommend the movies of the most fav actor
    Most_popular_actor = user_rated_lead_movies["actorName"].value_counts(ascending =False).head(1)


    d1= pd.DataFrame(user_rated_lead_movies.groupby("userID")["actorName"].value_counts(ascending =False))
    d1 = d1.rename({"actorName":"Number"},axis=1)
    d1 = d1.reset_index()
    Fav_actor = pd.merge(d1,d1.loc[d1.groupby('userID')["Number"].idxmax(),['userID','actorName']])


    # Recommending top ten movies on the basis of the predicted ratings by the user
    #UserID = int(input("Enter your userID : "))

    # unique MovieID throughout the dataset
    unique_iid = Final_data["movieID"].unique()

    # movies which the user has watched
    u_iid = Final_data[Final_data["userID"]==UserID]["movieID"]

    # movies which the user has not watched
    not_watched_iid = np.setdiff1d(unique_iid,u_iid)


    # movies not watched yet
    NW = Final_data[Final_data["movieID"].isin(list(not_watched_iid))][["title","actorName","avg_rating"]]
    NW.head()
    num_movies = int(input("# How many recommendations do you want :  " )) 
    if UserID in list(Fav_actor["userID"]):
        user_fav = list(Fav_actor[Fav_actor["userID"]==UserID]["actorName"])
    else:  
        user_fav = ["Tom Hanks"]


    # final recommendation
    rec_movies = NW[NW["actorName"]== user_fav[0]][["title","avg_rating"]]
    print("# The recommended movies are : ", pd.DataFrame(rec_movies.sort_values(by="avg_rating",ascending = False)["title"]).reset_index().drop("index",axis =1).drop_duplicates().reset_index().drop("index",axis =1).head(num_movies),sep = "\n")

### Director Function :

In [24]:
def director_based(UserID):


    # RECOMMENDATION Based On director
    # Dataframe of Fav director each user

    # if any user is not present in the data then we will recommend the movies of the most fav director
    Most_popular_director = Final_data["directorName"].value_counts(ascending =False).head(1)

    # Dataframe of Fav director each user
    d2= pd.DataFrame(Final_data.groupby("userID")["directorName"].value_counts(ascending =False))
    d2 = d2.rename({"directorName":"Number"},axis=1)
    d2 = d2.reset_index()
    Fav_director = pd.merge(d2,d2.loc[d2.groupby('userID')["Number"].idxmax(),['userID',"directorName"]])
    Fav_director.head()


    # Recommending top ten movies on the basis of the predicted ratings by the user
    #UserID = int(input("Enter your userID : "))

    # unique MovieID throughout the dataset
    unique_iid = Final_data["movieID"].unique()

    # movies which the user has watched
    u_iid = Final_data[Final_data["userID"]==UserID]["movieID"]

    # movies which the user has not watched
    not_watched_iid = np.setdiff1d(unique_iid,u_iid)


    # movies not watched yet
    NW = Final_data[Final_data["movieID"].isin(list(not_watched_iid))][["title","directorName","avg_rating"]]

    num_movies = int(input("# How many recommendations do you want : " )) 
    if UserID in list(Fav_director["userID"]):
        user_fav = list(Fav_director[Fav_director["userID"]==UserID]["directorName"])
    else:  
        user_fav = ["Robert Rodriguez"]


    # final recommendation
    rec_movies = NW[NW["directorName"]== user_fav[0]][["title","avg_rating"]]
    print("# The recommended movies are : ",pd.DataFrame(rec_movies.sort_values(by="avg_rating",ascending = False)["title"]).reset_index().drop("index",axis =1).drop_duplicates().reset_index().drop("index",axis =1).head(num_movies),sep = "\n")

### Genre Function :

In [25]:

#user defined function for getting 2 favourite genres of a particular user.
def users_fav2_genres(user_id):
    cols=['Action', 'Adventure', 'Animation', 'Children', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Thriller',
       'War', 'Western']
    tab=list(pd.DataFrame(final_merge1[final_merge1["userID"]==user_id][cols].sum().sort_values(ascending=False).head(2)).index)
    return tab


import random
def getting_recomm_movies(table,id1,num):
    #top 5 movies from these genres which user has not watched based on rating
    cols=['Action', 'Adventure', 'Animation', 'Children', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Thriller',
       'War', 'Western']
    tab2=final_merge1[final_merge1["userID"]!=id1][table]
    movie_genre1=list(tab2[tab2[table[0]]==1].groupby(["title"]).agg({"rating":np.mean}).sort_values(ascending=False,by="rating").head(5).reset_index()["title"])
    movie_genre2=list(tab2[tab2[table[1]]==1].groupby(["title"]).agg({"rating":np.mean}).sort_values(ascending=False,by="rating").head(5).reset_index()["title"])
    movie_genre1.extend(movie_genre2)
    
    #returning the shuffled list
    return random.sample(movie_genre1,num)




def genre_based(UserID):
    #driver function
    if UserID not in list(final_merge1["userID"]):
        print("User Not Found")
    
    else :
        
        UserId = UserID
        num=int(input("# How many recommendations do you want : "))
        tab=users_fav2_genres(UserId)

        tab.extend(["title","userID","rating"])
        top10_movies=getting_recomm_movies(tab,UserId,num)
        print("# The recommended movies are : ",pd.DataFrame(top10_movies ,columns=["title"] ),sep = "\n")


### Actor Name Search Function :

In [26]:
def Actor_name(UserID):
    user= UserID
    actor_needed= str(input("# Enter the Full Name of the Actor : "))
    num = int(input("# How many Recommendations do you want : "))
    subset_user_ratedmovies= user_ratedmovies4[user_ratedmovies4['userID']==user]
    
    ### Checking if User is present in the database or not
    if len(subset_user_ratedmovies)>=1:
        print("  ")
    else:
        print("Invalid User ID, please re-check")
        sys.exit("User Not Found")


    ### Finding all the Movies where the actor has played the Lead Role
    actor_mov= movie_actor4[movie_actor4['actorName']==actor_needed]

    ### Checking if Actor is present in the database or not
    if len(actor_mov)>=1:
        print("  ")
    else:
        print("There is no Actor by this Name, please re-check")
        sys.exit("Actor Not Found")



    ### Recommending Movies of the Actor which has not been watched by User Yet
    df_all = actor_mov.merge(subset_user_ratedmovies.drop_duplicates(), on=['movieID'],how='left', indicator=True)
    recc1= df_all[df_all['_merge']=='left_only']
    recc_movie= recc1[['movieID','actorName']]
    final_recc= pd.merge(recc_movie,user_ratedmovies4,left_on='movieID',right_on='movieID',how='left')
    
    

    ### List of Movies of the Actor which has not been watched by User Yet
    final_recc = final_recc.sort_values(by='rating',ascending=False)
    final_recc = pd.merge(final_recc,movies,left_on="movieID",right_on="id",how='left')
    final_recc.drop_duplicates('title',inplace=True,keep='first')
    

    ### Top 5  Movie Recommendations
    final_recc = final_recc.reset_index()
    print(pd.DataFrame(final_recc["title"].head(num)))

## Driver Function - Recommendation

In [32]:
UserID = int(input("Enter your userID : "))
print("# Recommendation based on","1. Director","2. Actor","3. Genre", "4. Search by Actor Name",sep="\n")

option = int(input("# Please choose your option : " ))

if option == 1 :
    director_based(UserID)
    
elif option == 2 :
    actor_based(UserID)
    
elif option == 3 :
    genre_based(UserID)
    
elif option == 4 :
    Actor_name(UserID)
    
else :
    print("Sorry, Invalid Input")

Enter your userID : 78
# Recommendation based on
1. Director
2. Actor
3. Genre
4. Search by Actor Name
# Please choose your option : 2
# How many recommendations do you want :  4
# The recommended movies are : 
                      title
0      Deconstructing Harry
1  Manhattan Murder Mystery
2                   Bananas
3    What's Up, Tiger Lily?
